In [1]:
!pip install sqlalchemy-bigquery

In [2]:
import geopandas as gpd
import pandas as pd
import glob
import os

os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
from siuba import *

import prep_data2 as prep_data

DATA_PATH = "./data/test/"

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/importlib/__init__.py:127: UserWarning: Obsolete pybigquery is installed, which is likely to
interfere with sqlalchemy_bigquery.
pybigquery should be uninstalled.
  return _bootstrap._gcd_import(name[level:], package, level)
E0405 19:40:48.837978529     322 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [3]:
from datetime import datetime

time0 = datetime.now()

# Read in local parquets
stops = pd.read_parquet(f"{DATA_PATH}stops.parquet")
trips = pd.read_parquet(f"{DATA_PATH}trips.parquet")
route_info = pd.read_parquet(f"{DATA_PATH}route_info.parquet")
routes = gpd.read_parquet(f"{DATA_PATH}routes.parquet")
latest_itp_id = pd.read_parquet(f"{DATA_PATH}latest_itp_id.parquet")

time1 = datetime.now()
print(f"Read in data: {time1-time0}")

Read in data: 0:00:01.386307


In [4]:
# Left only means in trips, but shape_id not found in shapes.txt
# right only means in routes, but no route that has that shape_id 
# We probably should keep how = "left"?
# left only means we can assemble from stop sequence?
routes1 = pd.merge(
        trips,
        routes,
        on = ["calitp_itp_id", "shape_id"],
        how = "left",
        validate = "m:1",
        indicator=True
    )


In [5]:
routes1._merge.value_counts()

both          7832
left_only      150
right_only       0
Name: _merge, dtype: int64

In [6]:
missing_shapes = (routes1[routes1._merge=="left_only"]
      .drop(columns = ["geometry", "_merge"])
      .reset_index(drop=True)
     )

missing_shapes.head(2)

,calitp_itp_id,route_id,shape_id
0,301,8,1232_shp
1,301,6,1205_shp


In [46]:
SELECTED_DATE = "2022-3-15"

trip_cols = ["calitp_itp_id", "route_id", "shape_id"]

dim_trips = (tbl.views.gtfs_schedule_dim_trips()
             # filter first to just the smaller set of IDs in missing_shapes
             >> filter(_.calitp_itp_id.isin(missing_shapes.calitp_itp_id))
             # Now find those shape_ids and trips associated
             >> filter(_.shape_id.isin(missing_shapes.shape_id))
             >> select(*trip_cols, _.trip_key)
             >> distinct()
            )

missing_trips = (
    tbl.views.gtfs_schedule_fact_daily_trips()
    >> filter(_.service_date == SELECTED_DATE, 
           _.is_in_service==True)
    >> select(_.trip_key, _.trip_id)
    >> inner_join(_, dim_trips, on = "trip_key")
    >> distinct()
    >> collect()
)


In [47]:
# Since there are multiple trips, we'll sort the same way, and keep the first one
group_cols = ["calitp_itp_id", "route_id", "shape_id"]
missing_trips2 = (missing_trips.sort_values(group_cols + ["trip_id"])
                  .drop_duplicates(subset=group_cols)
                  .reset_index(drop=True)
)

In [48]:
missing_trips2.head()

,trip_key,trip_id,calitp_itp_id,route_id,shape_id
0,2608222843436200649,10001020,4,18,shp-18-18
1,-4284942941671306430,13543020,4,21,shp-21-13
2,8481064942467690783,10843020,4,215,shp-215-67
3,-5710070409613492543,1044020,4,239,shp-239-57
4,-7953694877841381049,11882020,4,28,shp-28-09


In [51]:
len(missing_trips2)

104

In [52]:
stop_info_trips = (
    tbl.views.gtfs_schedule_dim_stop_times()
    >> filter(_.calitp_itp_id.isin(missing_trips2.calitp_itp_id))
    #>> filter(_.trip_id.isin(missing_trips2.trip_id))
    >> inner_join(_, 
                  missing_trips2, 
                  on = ["calitp_itp_id", "trip_id"]
                 )
    >> distinct()
    >> inner_join(_,
                  tbl.views.gtfs_schedule_dim_stops(), 
                  on = ["calitp_itp_id", "stop_id"])
    >> select(_.calitp_itp_id, _.trip_id, 
              _.route_id,
              _.stop_id, _.stop_sequence,
              _.stop_lon, _.stop_lat)
    >> distinct()
)

AttributeError: 'DataFrame' object has no attribute 'last_op'

In [10]:
stops_df = stop_info >> collect()

In [11]:
len(stops_df)

243160

In [12]:
# Somehow, getting back some multiple points for same trip_id, stop_id
stops_df = (stops_df.sort_values(["calitp_itp_id", "trip_id", "stop_id"])
            .drop_duplicates(subset=["calitp_itp_id", "trip_id", "stop_id"])
            .reset_index(drop=True)
           )

In [13]:
len(stops_df)

230281

In [14]:
# At this point, there are multiple trips associated with a route
# This is expected
# But, maybe after the line geometry is assembled, keep the longest distance trip for each route
stops_df.sort_values(["calitp_itp_id", "route_id", "trip_id", "stop_sequence"])[
    (stops_df.route_id=="18") & (stops_df.calitp_itp_id==4)
]

/tmp/ipykernel_322/1288799387.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  stops_df.sort_values(["calitp_itp_id", "route_id", "trip_id", "stop_sequence"])[


,calitp_itp_id,trip_id,route_id,stop_id,stop_sequence,stop_lon,stop_lat
11,4,10001020,18,1584,1,-122.086950,37.669892
18,4,10001020,18,2309,1,-122.265807,37.797160
92,4,10001020,18,50555,1,-122.086950,37.669892
164,4,10001020,18,55833,1,-122.265807,37.797160
10,4,10001020,18,1479,2,-122.083060,37.670918
...,...,...,...,...,...,...,...
200353,4,9395020,18,6065,59,-122.222245,37.773170
200179,4,9395020,18,2839,60,-122.225458,37.775518
200244,4,9395020,18,51769,60,-122.299531,37.884226
200309,4,9395020,18,55886,60,-122.225458,37.775518


In [15]:
# the edge cases are have route_id assigned to be shape_id
# let's get rid of multiple trips now
stops_df2 = (stops_df.sort_values(["calitp_itp_id", "route_id", "trip_id""])
             .drop_duplicates(subset=["calitp_itp_id", "route_id"])
             .reset_index(drop=True)
             # We don't have calitp_url_number
             # Add it here, but get rid of it later
             .assign(calitp_url_number=0)
            )

In [26]:
len(stops_df)

230281

In [27]:
len(stops_df2)

60

In [16]:
def make_routes_line_geom_for_missing_shapes(df, CRS="EPSG:4326"):
    """
    Parameters:
    df: pandas.DataFrame.
        Compile a dataframe from gtfs_schedule_trips.
        Find the trips that aren't in `shapes.txt`.
        https://github.com/cal-itp/data-analyses/blob/main/traffic_ops/create_routes_data.py#L63-L69
        Use that dataframe here.
    CRS: str, a projected coordinated reference system.
            Defaults to EPSG:4326 (WGS84)
    """
    if "shape_id" not in df.columns:
        df = df.assign(shape_id=df.route_id)

    # Make a gdf
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df.stop_lon, df.stop_lat),
        crs=WGS84,
    )

    # Count the number of stops for a given shape_id
    # Must be > 1 (need at least 2 points to make a line)
    gdf = gdf.assign(
        num_stops=(gdf.groupby("shape_id")["stop_sequence"].transform("count"))
    )

    # Drop the shape_ids that can't make a line
    gdf = (
        gdf[gdf.num_stops > 1]
        .reset_index(drop=True)
        .assign(stop_sequence=gdf.stop_sequence.astype(int))
        .drop(columns="num_stops")
    )

    # shapely make linestring with groupby
    # https://gis.stackexchange.com/questions/366058/pandas-dataframe-to-shapely-linestring-using-groupby-sortby
    group_cols = ["calitp_itp_id", "calitp_url_number", "shape_id"]

    gdf2 = (
        gdf.sort_values(group_cols + ["stop_sequence"])
        .groupby(group_cols)["geometry"]
        .apply(lambda x: shapely.geometry.LineString(x.tolist()))
    )

    gdf2 = gpd.GeoDataFrame(gdf2, geometry="geometry", crs=WGS84)
    gdf2 = (
        gdf2.to_crs(CRS)
        .sort_values(["calitp_itp_id", "calitp_url_number", "shape_id"])
        .drop_duplicates()
        .reset_index(drop=True)
    )

    return 

In [17]:
df = stops_df2.copy()

In [25]:
len(stops_df2)

60

In [23]:
import utils
WGS84=utils.WGS84

if "shape_id" not in df.columns:
    df = df.assign(shape_id=df.route_id)

# Make a gdf
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.stop_lon, df.stop_lat),
    crs=WGS84,
)

In [24]:
# Count the number of stops for a given shape_id
# Must be > 1 (need at least 2 points to make a line)
print(f"# obs to start: {len(gdf)}")
gdf = gdf.assign(
    num_stops=(gdf.groupby("shape_id")["stop_sequence"].transform("count"))
)

# Drop the shape_ids that can't make a line
gdf = (
    gdf[gdf.num_stops > 1]
    .reset_index(drop=True)
    .assign(stop_sequence=gdf.stop_sequence.astype(int))
    .drop(columns="num_stops")
)

print(f"# obs after trips with only 1 stop removed: {len(gdf)}")


# obs to start: 60
# obs after trips with only 1 stop removed: 8


In [21]:
gdf

,calitp_itp_id,trip_id,route_id,stop_id,stop_sequence,stop_lon,stop_lat,calitp_url_number,shape_id,geometry
0,4,10001020,18,1430,5,-122.078886,37.674847,0,18,POINT (-122.07889 37.67485)
1,4,10023020,40,1441,25,-122.095696,37.668723,0,40,POINT (-122.09570 37.66872)
2,4,10161020,6,2033,27,-122.274464,37.802963,0,6,POINT (-122.27446 37.80296)
3,296,3762421-202220D-vs20222D-Weekday-09,10,1341R,35,-122.053680,36.977638,0,10,POINT (-122.05368 36.97764)
4,296,3762844-202220D-vs20222D-Weekday-09,40,1520R,61,-122.194145,37.010872,0,40,POINT (-122.19415 37.01087)
5,301,t2E9-b134-slC,10,80100,4,-122.713715,38.438690,0,10,POINT (-122.71371 38.43869)
6,301,t3FC-b137-slC,18,80100,2,-122.713715,38.438690,0,18,POINT (-122.71371 38.43869)
7,301,t267-b12D-slC,6,80100,30,-122.713715,38.438690,0,6,POINT (-122.71371 38.43869)


In [ ]:
for i in range(len(gdf)):
    s = gdf.sort_values(["calitp_itp_id", "shape_id", "stop_sequence"])["shape_id"].iloc[i]
    print(s, type(s))

In [ ]:
gdf.drop_duplicates(subset=["calitp_itp_id", "shape_id", 
                            "stop_sequence"]).sort_values(
    ["calitp_itp_id", "shape_id"]
)[gdf.route_id=="18"].plot()

In [ ]:
group_cols = ["calitp_itp_id", "calitp_url_number", "shape_id"]
gdf2 = (
    gdf.sort_values(group_cols + ["stop_sequence"])
    .groupby(group_cols)["geometry"]
    .apply(lambda x: shapely.geometry.LineString(x.tolist()))
)


In [ ]:
gdf

In [ ]:
 gdf2 = gpd.GeoDataFrame(gdf2, geometry="geometry", crs=WGS84)
    gdf2 = (
        gdf2.to_crs(CRS)
        .sort_values(["calitp_itp_id", "calitp_url_number", "shape_id"])
        .drop_duplicates()
        .reset_index(drop=True)
    )

In [ ]:
import utils
import shapely

WGS84 = utils.WGS84
missing_line_geom = make_routes_line_geom_for_missing_shapes(stops_df2)

In [ ]:
#routes_part2 = prep_data.attach_route_name(stops_df, route_info)

In [ ]:
missing_shapes[["calitp_itp_id", "shape_id"]].drop_duplicates().shape

In [ ]:
# Loop through each operator
# otherwise, as soon as collect() is used, kernel crashes
all_missing_stops = pd.DataFrame()

missing_ids = list(missing_shapes.calitp_itp_id.unique())

for i in missing_ids:
    missing_stops = (
        missing_trips
        >> filter(_.calitp_itp_id == int(i))
        >> inner_join(_, stop_info,
                      on = ["calitp_itp_id", "trip_id"])
        >> collect()
        >> inner_join(_, missing_shapes, 
                      on = ["calitp_itp_id", "route_id", "shape_id"])
        )
    
    # Append the missing stops for each operator together
    all_missing_stops = pd.concat([all_missing_stops, missing_stops], 
                                  ignore_index=True)
    

In [ ]:
all_missing_stops.head()


In [ ]:
def routes_for_operators_in_shapes(routes, trips, route_info):
    # Attach route_id from gtfs_schedule.trips, using shape_id
    routes1 = pd.merge(
        routes,
        trips,
        on = ["calitp_itp_id", "shape_id"],
        # There are shape_ids that are left_only (1,600 obs)
        how = "inner",
        validate = "1:m",
    )

    routes_part1 = prep_data.attach_route_name(routes1, route_info)
    return routes_part1


In [ ]:
routes_part1 = routes_for_operators_in_shapes(routes, trips, route_info)

time2 = datetime.now()
print(f"Part 1 - routes for operator-routes in shapes.txt: {time2-time1}")

In [ ]:
def grab_missing_stops(ITP_ID, missing_trips_df, stops_df):
    df = (
        tbl.gtfs_schedule.stop_times()
        # Loop through individual operators, then do the join to find those trips
        # until the issue that makes kernel restarts is fixed
        >> filter(_.calitp_itp_id == ITP_ID)
        >> select(_.calitp_itp_id, _.stop_id, _.stop_sequence, _.trip_id)
        # Can't do isin without the collect()
        # But collect() is what is making kernel restart / shutting down notebook
        >> distinct()
        >> collect()
        >> inner_join(_, missing_trips_df, ["calitp_itp_id", "trip_id"])
        >> inner_join(_, stops_df, ["calitp_itp_id", "stop_id"])
    )
    
    return df








In [ ]:
def routes_for_operators_notin_shapes(routes_part1, stops, route_info):
    # Find the stops that aren't in `shapes.txt`
    missing_trips = (
        tbl.gtfs_schedule.trips()
        >> select(_.calitp_itp_id, _.route_id, _.shape_id, _.trip_id)
        >> distinct()
        >> collect()
        >> filter(~_.shape_id.isin(routes_part1.shape_id))
    )
    
    missing_trips_stops = grab_missing_stops(ITP_ID, missing_trips, stops)

        
    # Rename colums to match what's used in geography_utils
    missing_trips_stops = missing_trips_stops.rename(
        columns = {"stop_lon": "shape_pt_lon", 
                  "stop_lat": "shape_pt_lat",
                   "stop_sequence": "shape_pt_sequence",
                  }
    )
    
    missing_trips_stops2 = utils.make_routes_shapefile(
        LOOP_ME, CRS = utils.WGS84, 
        alternate_df=missing_trips_stops
    )
    
    missing_trips_stops2.to_parquet(f"{DATA_PATH}missing_routes.parquet")

    routes2 = pd.merge(
        missing_trips_stops2,
        (missing_trips_stops[["calitp_itp_id", "route_id", "shape_id"]]
         .assign(shape_id = missing_trips_stops.route_id)
        .drop_duplicates()
        ),
        on = ["calitp_itp_id", "shape_id"],
        how = "inner",
        validate = "1:m",
    )

    # In geography_utils.make_routes_shapefile, when alternate_df is set,
    # the shape_id is replaced with route_id (since shape_id is None)
    # Change it back after route line geometry created
    routes2 = routes2.assign(shape_id = None)
    
    routes_part2 = prep_data.attach_route_name(routes2, route_info)
    return routes_part2

In [ ]:
routes_part2 = routes_for_operators_notin_shapes(routes_part1, stops, route_info)
    
time3 = datetime.now()
print(f"Part 2 - routes for operator-routes not in shapes.txt: {time3-time2}")


In [ ]:
routes_assembled = (routes_part1.append(routes_part2)
                    .sort_values(["calitp_itp_id", "route_id"])
                    .drop_duplicates()
                    .reset_index(drop=True)
                   )

In [ ]:
# Attach agency_id and agency_name using calitp_itp_id
routes_assembled2 = prep_data.attach_agency_info(routes_assembled, agencies)

routes_assembled2 = (prep_data.filter_latest_itp_id(routes_assembled2, 
                                                    latest_itp_id, 
                                                    itp_id_col = "calitp_itp_id")
                     # Any renaming to be done before exporting
                     .rename(columns = prep_data.RENAME_COLS)
                     .sort_values(["itp_id", "route_id"])
                     .reset_index(drop=True)
                    )

print(f"Routes script total execution time: {time3-time0}")